### Importing libraries...

In [1]:
import requests
from pymongo import MongoClient
import pandas as pd
import re
import json
from pandas.io.json import json_normalize
import os
import numpy as np
import math
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap

### Importing Companies Data

In [2]:
dbName = "companies.json"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

### Verifying amount of companies of potential chosen country

In [3]:
data = db.companies.aggregate([{"$group":{"_id":"$offices.country_code","count":{"$sum":1}}}])
countries = list(data)

In [4]:
county_counts=[]
for x in countries:
    county_counts.append(x['_id'])


In [5]:
county_counts = [c for n in county_counts for c in n]


In [6]:
from collections import Counter
county_counts = Counter(county_counts)
{k: v for k, v in sorted(county_counts.items(), key=lambda item: item[1], reverse=True)}

{'USA': 775,
 'GBR': 257,
 'DEU': 116,
 'IND': 106,
 'FRA': 82,
 'AUS': 80,
 'CHN': 79,
 'ESP': 62,
 'CAN': 62,
 'JPN': 52,
 'SGP': 46,
 'NLD': 40,
 'SWE': 35,
 'CHE': 31,
 'BRA': 30,
 'ARG': 29,
 'ITA': 28,
 'ISR': 28,
 'IRL': 25,
 'HKG': 25,
 'RUS': 23,
 'TWN': 20,
 'BEL': 19,
 'MEX': 18,
 'NOR': 17,
 'DNK': 16,
 'PHL': 15,
 'ZAF': 15,
 'NZL': 15,
 'ROM': 15,
 'KOR': 15,
 'MYS': 14,
 'AUT': 14,
 'ARE': 14,
 'POL': 13,
 'UKR': 12,
 'FIN': 12,
 'PRT': 12,
 'TUR': 11,
 'CZE': 11,
 'CHL': 9,
 'VNM': 9,
 'BGR': 8,
 'GRC': 7,
 'THA': 6,
 'COL': 6,
 'SVN': 6,
 'EST': 5,
 'HUN': 5,
 'ANT': 5,
 'EGY': 5,
 'LVA': 4,
 'LKA': 4,
 'PAK': 4,
 'HRV': 4,
 'CYP': 4,
 'BGD': 3,
 'NGA': 3,
 'IDN': 3,
 'SAU': 3,
 'URY': 3,
 'LTU': 3,
 'MAR': 3,
 'JOR': 3,
 'MLT': 3,
 'LUX': 3,
 'MKD': 3,
 'VEN': 2,
 'PAN': 2,
 'GIB': 2,
 'LBN': 2,
 'ISL': 2,
 'MUS': 2,
 'BIH': 2,
 'CSS': 2,
 'NIC': 2,
 'PRK': 2,
 'QAT': 2,
 'UZB': 2,
 'KEN': 2,
 'VGB': 2,
 'ARA': 2,
 'PER': 1,
 'SDN': 1,
 'DOM': 1,
 'TUN': 1,
 'ALB': 1,

**Countries which companies I have more information of:**  
  
USA 🇺🇸  
GBR 🇬🇧  
DEU 🇩🇪   
IND 🇮🇪  
FRA 🇫🇷  

**Top Videogame development Countries:**

USA 🇺🇸 = 194  
GBR 🇬🇧 =  48  
DEU 🇩🇪 =   /  
IND 🇮🇪  =   /  
FRA 🇫🇷 =  23  

**Chosen Country:  
US** 🇺🇸 🇺🇸 🇺🇸 🇺🇸

## Location

**Requisite:**  
Developers like to be near successful tech startups that have raised at least 1 Million dollars.

**Query:**  
 - Country: USA  
 - Is a startup: founded after 2007 
 - Tech categories: software, design, games_video, web, mobile
 - Raised at least 1 Million dollars 
 

In [7]:
query = {"$and":[
    {"offices.country_code":"USA"},
    {"founded_year":{"$gte":2007}},
    {"funding_rounds.raised_amount":{"$gte":1000000}},
    {"category_code":{"$in":["software","design","games_video","web", "mobile"]}},
]}
cur = db.companies.find(query)
df = pd.DataFrame(list(cur))
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297da9,Mahalo,mahalo,http://www.crunchbase.com/company/mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40.0,...,[],None,[],"[{'description': '', 'address1': '3525 Eastham...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...",[],[],[]
1,52cdef7c4bab8bd675297dc2,AdaptiveBlue,adaptiveblue,http://www.crunchbase.com/company/adaptiveblue,http://www.getglue.com,http://blog.adaptiveblue.com,http://blog.adaptiveblue.com/?feed=rss2,GetGlue,games_video,15.0,...,[],None,[],"[{'description': '', 'address1': '131 Varick S...",[],None,"[{'embed_code': '<object width=""425"" height=""3...",[],[{'external_url': 'http://web20show.com/2009/1...,[]
2,52cdef7c4bab8bd675297dc0,Livestream,livestream,http://www.crunchbase.com/company/livestream,http://www.livestream.com,http://www.livestream.com/blog/,http://www.livestream.com/blog/?feed=rss2,livestream,games_video,120.0,...,[],"{'price_amount': None, 'price_currency_code': ...",[],"[{'description': 'Livestream HQ', 'address1': ...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...",[],[{'external_url': 'http://scobleizer.com/2007/...,[]
3,52cdef7c4bab8bd675297dc1,Ustream,ustream,http://www.crunchbase.com/company/ustream,http://www.ustream.tv,http://www.ustream.com/blog,http://www.ustream.com/blog/feed,USTREAM,games_video,250.0,...,[],None,[],"[{'description': 'San Francisco Office', 'addr...","[{'id': 6587, 'description': 'Ustream recogniz...",None,"[{'embed_code': '<object id=""flashObj"" width=""...","[{'available_sizes': [[[150, 103], 'assets/ima...",[{'external_url': 'http://facebook.com/ustream...,[]
4,52cdef7c4bab8bd675297dd1,SodaHead,sodahead,http://www.crunchbase.com/company/sodahead,http://sodahead.com,,,SodaHead,web,25.0,...,[],None,[],"[{'description': '', 'address1': '16161 Ventur...",[],None,[],"[{'available_sizes': [[[150, 131], 'assets/ima...",[{'external_url': 'http://www.prweb.com/releas...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,52cdef7f4bab8bd67529c4f0,Culture Jam,culture-jam,http://www.crunchbase.com/company/culture-jam,http://site.promojam.com,http://promojam.tumblr.com/,,culturejam,web,15.0,...,[],None,[],"[{'description': '', 'address1': '2434 Lincoln...",[],None,[],[],[{'external_url': 'http://www.techcrunch.com/2...,[]
295,52cdef7f4bab8bd67529c55b,Clavis Technology,clavis-technology,http://www.crunchbase.com/company/clavis-techn...,http://www.clavistechnology.com,http://www.clavistechnology.com/news,http://www.clavistechnology.com/feed/,Clavistech,software,50.0,...,[],None,[],"[{'description': '', 'address1': '7th Floor,',...","[{'id': 7702, 'description': 'Clavis Secures S...",None,"[{'embed_code': '<p> <object classid=""clsid:d...","[{'available_sizes': [[[150, 92], 'assets/imag...",[{'external_url': 'http://the451group.com/repo...,[]
296,52cdef7f4bab8bd67529c59b,Luma International,luma-international,http://www.crunchbase.com/company/luma-interna...,http://www.lumacentral.com,http://blog.lumacentral.com,http://blog.lumacentral.com/atom.xml,lumacentral,software,40.0,...,[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],None,[],"[{'available_sizes': [[[150, 114], 'assets/ima...",[],[]
297,52cdef7f4bab8bd67529c5b8,Continuity Engine,continuity-engine,http://www.crunchbase.com/company/continuity-e...,http://www.continuity.net,http://continuityengine.blogspot.com/,,ContinuityEng,software,11.0,...,[],None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'HQ', 'address1': '150 Munson...",[],None,[],"[{'available_sizes': [[[150, 72], 'assets/imag...",[],[]


In [8]:
df.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'ipo', 'video_embeds',
       'screenshots', 'external_links', 'partners'],
      dtype='object')

In [9]:
offices = df[["name", "offices"]].explode("offices")
offices

,name,offices
0,Mahalo,"{'description': '', 'address1': '3525 Eastham ..."
1,AdaptiveBlue,"{'description': '', 'address1': '131 Varick St..."
2,Livestream,"{'description': 'Livestream HQ', 'address1': '..."
3,Ustream,"{'description': 'San Francisco Office', 'addre..."
3,Ustream,"{'description': 'Los Angeles Office', 'address..."
...,...,...
296,Luma International,"{'description': 'HQ North America', 'address1'..."
296,Luma International,"{'description': 'Office France', 'address1': '..."
296,Luma International,"{'description': 'Office The Netherlands', 'add..."
297,Continuity Engine,"{'description': 'HQ', 'address1': '150 Munson ..."


In [10]:
offices = json_normalize(json.loads(offices.to_json(orient="records")))
offices = offices[["name", "offices.latitude", "offices.longitude", "offices.state_code"]]
offices

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,offices.latitude,offices.longitude,offices.state_code
0,Mahalo,34.017606,-118.487267,CA
1,AdaptiveBlue,40.801358,-74.337200,NY
2,Livestream,40.726155,-73.995625,NY
3,Ustream,37.392936,-122.079480,CA
4,Ustream,NaN,NaN,CA
...,...,...,...,...
388,Luma International,NaN,NaN,CA
389,Luma International,NaN,NaN,None
390,Luma International,NaN,NaN,None
391,Continuity Engine,41.327918,-73.243552,CT


In [11]:
offices["offices.state_code"].value_counts()

CA    184
NY     50
MA     23
WA     11
TX      9
CO      9
VA      7
FL      5
IL      5
NC      5
PA      4
OR      3
CT      3
GA      3
UT      3
AZ      2
NJ      2
MN      2
MD      2
DC      1
LA      1
SC      1
NH      1
ME      1
RI      1
KY      1
Name: offices.state_code, dtype: int64

### Chosen state: California

In [12]:
offices = offices.loc[offices["offices.state_code"] == "CA"]
offices = offices.dropna()
offices = offices.reset_index()
offices = offices.drop(columns = ["index"])

In [13]:
offices

,name,offices.latitude,offices.longitude,offices.state_code
0,Mahalo,34.017606,-118.487267,CA
1,Ustream,37.392936,-122.079480,CA
2,SodaHead,37.269175,-119.306607,CA
3,Funny Or Die,37.421621,-122.137444,CA
4,Seesmic,37.775196,-122.419204,CA
...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA
139,CrowdGather,34.168955,-118.575293,CA
140,High Gear Media,37.445335,-122.158330,CA
141,Teambox,41.406335,2.192023,CA


In [14]:
offices = offices.rename(columns={"name": "Name", "offices.latitude": "Latitude", "offices.longitude": "Longitude", "offices.state_code": "State Code"})

In [15]:
offices

,Name,Latitude,Longitude,State Code
0,Mahalo,34.017606,-118.487267,CA
1,Ustream,37.392936,-122.079480,CA
2,SodaHead,37.269175,-119.306607,CA
3,Funny Or Die,37.421621,-122.137444,CA
4,Seesmic,37.775196,-122.419204,CA
...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA
139,CrowdGather,34.168955,-118.575293,CA
140,High Gear Media,37.445335,-122.158330,CA
141,Teambox,41.406335,2.192023,CA


In [16]:
m = Map(location=[37.794020, -122.403124],zoom_start=4)
for i, row in offices.iterrows():
    icon = Icon(
            color="red",
            prefix="fa",
            icon="trophy")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
m

In [17]:
#Eliminate not in California: 
intruders = ["GENWI", "Zinch", "Filtrbox", "Titan Gaming", "DotBlu", "Formspring", "ChessPark", "Socialize", "Fortumo", "Zendesk", "XIHA", "Teambox"]
for name in intruders: 
    offices = offices[offices.Name != name]

In [18]:
m = Map(location=[35.794020, -120.403124],zoom_start=7)
for i, row in offices.iterrows():
    icon = Icon(
            color="red",
            prefix="fa",
            icon="fa-building")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
m

## Google API

In [19]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_key")
url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=starbucks+in+CA&key={apiKey}"
res = requests.get(url)


In [20]:
res = res.json()

In [21]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_key")
url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
#query=starbucks+in+CA&key={apiKey}&pageToken=res['next_page_token']"
res = requests.get(url)
res = res.json()

In [22]:
def getData(query, page_token=None):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {"key":os.getenv("google_key"), "query":query}
    if page_token:
        params["pageToken"]=page_token
    res = requests.get(url, params=params)
    res = res.json()
    results = res["results"]
    if 'next_page_token' in res.keys():
        page_token = res['next_page_token']
        results=[results, page_token]
    else: 
        results=[results,False]
    return results

In [23]:
def getLocations(query, limit=50):
    data = []
    page_token = None
    while len(data)<limit:
        results, page_token = getData(query, page_token)
        data+=results
        if not page_token: 
            break
    return data

##  Function to calculate distances in km

**Latitude:** 1 deg = 110.574 km.   
**Longitude:** 1 deg = 111.320*cos(latitude) km.

In [24]:
'''
def FindDistances(req):
    d=[]
    for i in offices.index:
        d_list=[]
        for ind in req.index: 
            x = req['Latitude'][ind]
            y = req['Longitude'][ind] 
            distance = (((x-(offices['Latitude'][i]))**2)+((y-(offices['Longitude'][i]))**2))**(1/2)
            d_list.append(distance)
        d.append(min(d_list))
    offices['New'] = d
'''

"\ndef FindDistances(req):\n    d=[]\n    for i in offices.index:\n        d_list=[]\n        for ind in req.index: \n            x = req['Latitude'][ind]\n            y = req['Longitude'][ind] \n            distance = (((x-(offices['Latitude'][i]))**2)+((y-(offices['Longitude'][i]))**2))**(1/2)\n            d_list.append(distance)\n        d.append(min(d_list))\n    offices['New'] = d\n"

In [25]:
def FindDistances(req):
    d=[]
    for i in offices.index:
        d_list=[]
        for ind in req.index: 
            x = req['Latitude'][ind]*110.574
            y = req['Longitude'][ind]*111.320*math.cos(req['Latitude'][ind]) 
            distance = (((x-(offices['Latitude'][i]*110.574))**2)+((y-(offices['Longitude'][i]*111.320*math.cos(offices['Latitude'][i])))**2))**(1/2)
            d_list.append(distance)
        d.append(min(d_list))
    offices['New'] = d

## Starbucks

**Requisite:**  
Executives like Starbucks A LOT. Ensure there's a starbucks not to far.

In [26]:
data = getLocations("starbucks in CA", 100)
dataset = pd.DataFrame(data)

In [27]:
starbucks = []
for st in dataset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= st['location']['lat']
    new_dict["Longitude"]= st['location']['lng']
    starbucks.append(new_dict)

In [28]:
starbucks = pd.DataFrame(starbucks)
starbucks

,Latitude,Longitude
0,34.072428,-117.881891
1,37.291159,-121.989834
2,37.762265,-121.433395
3,38.714409,-120.839673
4,38.762384,-120.576595
...,...,...
95,38.013442,-121.868315
96,34.890722,-116.999380
97,36.627799,-119.683340
98,37.969738,-121.289320


In [29]:
def transformToGeoPoint(s):
    if np.isnan(s.Latitude) or np.isnan(s.Longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.Longitude, s.Latitude]
    }
starbucks["geopoint"] = starbucks.apply(transformToGeoPoint, axis=1)
starbucks

,Latitude,Longitude,geopoint
0,34.072428,-117.881891,"{'type': 'Point', 'coordinates': [-117.881891,..."
1,37.291159,-121.989834,"{'type': 'Point', 'coordinates': [-121.9898337..."
2,37.762265,-121.433395,"{'type': 'Point', 'coordinates': [-121.4333954..."
3,38.714409,-120.839673,"{'type': 'Point', 'coordinates': [-120.839673,..."
4,38.762384,-120.576595,"{'type': 'Point', 'coordinates': [-120.5765954..."
...,...,...,...
95,38.013442,-121.868315,"{'type': 'Point', 'coordinates': [-121.868315,..."
96,34.890722,-116.999380,"{'type': 'Point', 'coordinates': [-116.9993796..."
97,36.627799,-119.683340,"{'type': 'Point', 'coordinates': [-119.68334, ..."
98,37.969738,-121.289320,"{'type': 'Point', 'coordinates': [-121.28932, ..."


In [30]:
FindDistances(starbucks)

In [31]:
offices = offices.rename(columns={"New":"Starbucks"})

In [32]:
m = Map(location=[35.794020, -120.403124],zoom_start=7)
for i, row in offices.iterrows():
    icon = Icon(
            color="red",
            prefix="fa",
            icon="fa-building")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
    
for i, row in starbucks.iterrows():
    icon = Icon(
            color="green",
            prefix="fa",
            icon="fa-coffee")
    Marker(
        location=row[["Latitude","Longitude"]],
        icon=icon
    ).add_to(m)
m

## Airports

**Requisite:**  
Account managers need to travel a lot.

In [33]:
data = getLocations("Airport in California", 100)
airportset = pd.DataFrame(data)

In [34]:
airportset

,business_status,formatted_address,geometry,icon,id,name,photos,place_id,plus_code,rating,reference,types,user_ratings_total,opening_hours
0,OPERATIONAL,"1 World Way, Los Angeles, CA 90045, United States","{'location': {'lat': 33.9415889, 'lng': -118.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,2b1ad644609c86ef8156eeec48a4675d580224d9,Los Angeles International Airport,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJtU-yE9KwwoAR8a2LaVd7qHc,"{'compound_code': 'WHRR+JH Los Angeles, Califo...",3.6,ChIJtU-yE9KwwoAR8a2LaVd7qHc,"[airport, point_of_interest, establishment]",12166,NaN
1,OPERATIONAL,"San Francisco, CA 94128, United States","{'location': {'lat': 37.6213129, 'lng': -122.3...",https://maps.gstatic.com/mapfiles/place_api/ic...,9c9fb107d7a776c3f6a26791b07982d7db079113,San Francisco International Airport,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJVVVVVYx3j4ARP-3NGldc8qQ,{'compound_code': 'JJCC+GC South San Francisco...,4.0,ChIJVVVVVYx3j4ARP-3NGldc8qQ,"[airport, point_of_interest, establishment]",7400,NaN
2,OPERATIONAL,"18601 Airport Way, Santa Ana, CA 92707, United...","{'location': {'lat': 33.6761901, 'lng': -117.8...",https://maps.gstatic.com/mapfiles/place_api/ic...,da9ed12835b3a597d4bcbc961a779ab353b0d145,John Wayne Airport,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ70Cry-3e3IAR4hkRw_XoQck,"{'compound_code': 'M4GM+F2 Santa Ana, Californ...",4.4,ChIJ70Cry-3e3IAR4hkRw_XoQck,"[airport, point_of_interest, establishment]",1583,{'open_now': True}
3,OPERATIONAL,"Ontario, CA 91761, United States","{'location': {'lat': 34.0559781, 'lng': -117.5...",https://maps.gstatic.com/mapfiles/place_api/ic...,ec4d4162373ff66e9c6628eec8b410c40019d703,Ontario International Airport,"[{'height': 3021, 'html_attributions': ['<a hr...",ChIJp9TPWvw0w4ARO1JZyBbtxJ8,"{'compound_code': '3C42+9Q Ontario, California...",4.3,ChIJp9TPWvw0w4ARO1JZyBbtxJ8,"[airport, point_of_interest, establishment]",1254,{'open_now': True}
4,OPERATIONAL,"6900 Airport Blvd, Sacramento, CA 95837, Unite...","{'location': {'lat': 38.6950854, 'lng': -121.5...",https://maps.gstatic.com/mapfiles/place_api/ic...,b825239c5bd3448da42434d6402297b961b445a2,Sacramento International Airport,"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJrXBqznMrm4ARMkLvKBKQYKQ,"{'compound_code': 'MCW5+2X Sacramento, Califor...",4.1,ChIJrXBqznMrm4ARMkLvKBKQYKQ,"[airport, point_of_interest, establishment]",1541,{'open_now': True}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,OPERATIONAL,"12653 Osborne St, Pacoima, CA 91331, United St...","{'location': {'lat': 34.2598054, 'lng': -118.4...",https://maps.gstatic.com/mapfiles/place_api/ic...,a27abbb55ffe476125745b8c7eed9c0eac4c7823,Whiteman Airport,"[{'height': 1489, 'html_attributions': ['<a hr...",ChIJAAAAAAAAAAARk0sL96tOFRM,"{'compound_code': '7H5Q+W6 Pacoima, Los Angele...",4.3,ChIJAAAAAAAAAAARk0sL96tOFRM,"[airport, point_of_interest, establishment]",43,NaN
96,OPERATIONAL,"3400 E Tahquitz Canyon Way, Palm Springs, CA 9...","{'location': {'lat': 33.8303194, 'lng': -116.5...",https://maps.gstatic.com/mapfiles/place_api/ic...,2f3d34371900b038e82bf59a7ac235eaff50802a,Palm Springs International Airport,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJexz7C3wc24ARkCj0M00Pwtw,"{'compound_code': 'RFJV+45 Palm Springs, Calif...",4.2,ChIJexz7C3wc24ARkCj0M00Pwtw,"[airport, point_of_interest, establishment]",488,NaN
97,OPERATIONAL,"500 James Fowler Rd, Santa Barbara, CA 93117, ...","{'location': {'lat': 34.4271935, 'lng': -119.8...",https://maps.gstatic.com/mapfiles/place_api/ic...,2d04db686a2c2a47941cc5841bd491ecd3b5a34e,Santa Barbara Airport,"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJJ79bfX4_6YARXEGp_YImp6A,"{'compound_code': 'C5G6+V2 Santa Barbara, Cali...",4.4,ChIJJ79bfX4_6YARXEGp_YImp6A,"[airport, point_of_interest, establishment]",194,{'open_now': True}
98,OPERATIONAL,"550 Sally Ride Dr, Concord, CA 94520, United S...","{'location': {'lat': 37.9902157, 'lng': -122.0...",ht

In [35]:
airports = []
for a in airportset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= a['location']['lat']
    new_dict["Longitude"]= a['location']['lng']
    airports.append(new_dict)

In [40]:
airports = pd.DataFrame(airports)

In [41]:
FindDistances(airports)

In [42]:
offices = offices.rename(columns={"New":"Airports"})

In [43]:
offices

,Name,Latitude,Longitude,State Code,Starbucks,Airports,Airports
0,Mahalo,34.017606,-118.487267,CA,85.761785,165.047509,165.047509
1,Ustream,37.392936,-122.079480,CA,82.697703,87.747219,87.747219
2,SodaHead,37.269175,-119.306607,CA,392.961334,745.426042,745.426042
3,Funny Or Die,37.421621,-122.137444,CA,20.080443,87.311606,87.311606
4,Seesmic,37.775196,-122.419204,CA,8.611255,17.424266,17.424266
...,...,...,...,...,...,...,...
137,Moviepilot,34.052234,-118.243685,CA,90.708403,39.248205,39.248205
138,Global Fitness Media,34.054318,-118.462876,CA,56.088814,73.874109,73.874109
139,CrowdGather,34.168955,-118.575293,CA,121.956467,118.863944,118.863944
140,High Gear Media,37.445335,-122.158330,CA,4.297714,159.308290,159.308290


In [45]:
m = Map(location=[35.794020, -120.403124],zoom_start=7)
for i, row in offices.iterrows():
    icon = Icon(
            color="red",
            prefix="fa",
            icon="fa-building")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
    
for i, row in airports.iterrows():
    icon = Icon(
            color="blue",
            prefix="fa",
            icon="fa-plane")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
    
m

## Schools

In [46]:
data = getLocations("School in California", 100)
schoolset = pd.DataFrame(data)

In [47]:
schools = []
for s in schoolset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= s['location']['lat']
    new_dict["Longitude"]= s['location']['lng']
    schools.append(new_dict)

In [53]:
schools = pd.DataFrame(schools)
schools

,Latitude,Longitude
0,40.476588,-3.853554
1,33.878099,-118.054286
2,33.861031,-118.254754
3,33.688892,-117.925587
4,33.805531,-117.841449
...,...,...
95,32.882748,-117.222179
96,34.123312,-117.738770
97,37.848269,-122.239620
98,33.608324,-117.268230


In [54]:
FindDistances(schools)

In [55]:
offices = offices.rename(columns={"New":"Schools"})

In [60]:
m = Map(location=[35.794020, -120.403124],zoom_start=7)
for i, row in offices.iterrows():
    icon = Icon(
            color="red",
            prefix="fa",
            icon="fa-building")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
    
for i, row in schools.iterrows():
    icon = Icon(
            color="orange",
            prefix="fa",
            icon="fa-child")
    Marker(
        location=row[["Latitude","Longitude"]],
        icon=icon
    ).add_to(m)
    
m

## Average of distances

**Importance**  
  
Considering the employees will have to take a cab to go to the airport or take their kids to school.   
It would be nice to have a starbucks at a walking distance, therefore I will get rid of any offices which's distance to a starbucks or a bar is greater than 


In [55]:
offices["Sum of Distances"] = offices["Starbucks"]+offices["Airports"]+offices["Schools"]

In [56]:
offices.sort_values(by=["Starbucks"]).head(20)

,Name,Latitude,Longitude,State Code,Airports,Starbucks,Schools,Sum of Distances
100,PicApp,37.794020,-122.403124,CA,25.851200,0.344284,7.570147,33.765631
7,DanceJam,37.781557,-122.407959,CA,17.814970,0.663812,21.470440,39.949223
24,Gydget,37.791733,-122.398398,CA,24.103978,0.728794,9.335027,34.167799
45,Elastra,37.791137,-122.391843,CA,24.038719,0.763823,9.319617,34.122159
14,Crunchyroll,37.781265,-122.393229,CA,17.965814,1.051495,20.178780,39.196089
102,Reframe It,37.782163,-122.400591,CA,18.098246,1.178100,20.008070,39.284416
8,Yola,37.791115,-122.395475,CA,23.770442,1.186358,9.689477,34.646276
71,Tapulous,37.444332,-122.161018,CA,156.447486,1.365390,259.742299,417.555175
42,99designs,37.795531,-122.400598,CA,27.531949,1.912817,6.253077,35.697843
130,Milo,37.444531,-122.163352,CA,157.300434,2.179767,260.675778,420.155979


In [ ]:
m = Map(location=[35.794020, -120.403124],zoom_start=7)
for i, row in offices.iterrows():
    icon = Icon(
            color="red",
            prefix="fa",
            icon="fa-building")
    Marker(
        location=row[["Latitude","Longitude"]],
        tooltip=row["Name"],
        icon=icon
    ).add_to(m)
    
for i, row in schools.iterrows():
    icon = Icon(
            color="orange",
            prefix="fa",
            icon="fa-child")
    Marker(
        location=row[["Latitude","Longitude"]],
        icon=icon
    ).add_to(m)
    
m

# Chosen Office:   
  
### PicApp: ( 37.794020, -122.403124)

In [65]:
m = Map(location=[37.794020, -122.403124],zoom_start=5.5)
m

In [ ]:
m = Map(location=[37.794020, -122.403124],zoom_start=4)
for i, row in capitals.iterrows():
    if row["population"] > 2e5: 
        icon = Icon(
                color="red",
                prefix="fa",
                icon="trophy"
            )
    else:
        icon = Icon(
                color="green",
                prefix="fa",
                icon="paw"
            )
    Marker(
        location=row[["latitude","longitude"]],
        tooltip=row["state"],
        popup=row["capital"],
        icon=icon
    ).add_to(m)
m